# Adding new features to py4vasp

The following helper function is useful to look at the code of py4vasp

In [ ]:
import inspect
from IPython.display import HTML
from pygments import highlight
from pygments.lexers import PythonLexer
from pygments.formatters import HtmlFormatter

def show_code(obj):
    "Equivalent to print(inspect.getcode(obj)) but with syntax highlighting"
    code = inspect.getsource(obj)
    html = highlight(code, PythonLexer(), HtmlFormatter())
    css = HtmlFormatter().get_style_defs()
    return HTML(f"<style>{css}</style>{html}")

## Create a demo HDF5 file

In [ ]:
import numpy as np
import h5py

Let's generate the energies of an ionic relaxation

In [ ]:
labels = ("electronic", "ionic")
steps = np.arange(50)
electronic = np.exp(-0.1 * steps)
ionic = 2 * np.exp(-0.2 * steps)

and store it in an HDF5 file. We need to store the version, too, because py4vasp checks whether the version is compatible.

In [ ]:
with h5py.File("vaspout.h5", "w") as h5f:
    h5f["version/major"] = 7
    h5f["version/minor"] = 0
    h5f["version/patch"] = 0
    h5f["feature/energies"] = np.array([electronic, ionic]).T
    h5f["feature/labels"] = labels

At the end of this introduction, you will know how to add the content to py4vasp and produce a plot like the one below. 

In [ ]:
from py4vasp import plot
(
    plot(steps, electronic, "electronic", xlabel="step", ylabel="energy")
    + plot(steps, ionic, "ionic")
)

## Simple case: add new source for existing quantity

In this example, we add a new source to the existing Energy class

In [ ]:
from py4vasp import raw
show_code(raw.Energy)

by adding the following code to the [schema definition file](../src/py4vasp/_raw/definition.py).
~~~python
schema.add(
    raw.Energy,
    name="feature",
    labels="feature/labels",
    values="feature/energies",
)
~~~
plotting the new energy already works. Mind that you need to restart the Python kernel so that the changes to py4vasp are imported.

In [ ]:
from py4vasp import Calculation
calc = Calculation.from_path(".")
calc.energy[:].plot("feature(electronic) feature(ionic)")

In [ ]:
calc.band.plot(selection="kpoints_opt(s)")

The selection string _feature_ indicates that we want to read from the new quantity. It must match with the name that you used to add it to the schema.

## General case: Adding a new quantity that does not exist yet

### Introducing the raw quantity

py4vasp by convenction matches refinement classes to the data sources with the same name. Hence, if you want to introduce a functionality like
~~~python
calc.feature.plot()
~~~
you need to introduce a new raw.Feature class first [data.py](../src/py4vasp/_raw/data.py). We will use this simple definition
~~~python
@dataclasses.dataclass
class Feature:
    labels: VaspData
    values: VaspData
~~~
Note, that the _dataclass_ automatically adds the required constructors, getters, and setters.

The next step is to add the quantity to [the schema definition](../src/py4vasp/_raw/definition.py).
~~~python
schema.add(
    raw.Feature,
    required=raw.Version(7, 0),
    labels="feature/labels",
    values="feature/energies",
)
~~~
Here, we didn't use the _name_ argument because this intended to be the default argument.

This definition introduces the argument _required_. You can use this to specify that this feature is only available starting from a specific VASP version. I recommend this to avoid unnecessary questions in the forum and as a gentle reminder for users to upgrade. If the HDF5 file was produced by an older version of VASP, py4vasp will report an error like.
~~~
OutdatedVaspVersion: The energy is not available in VASP version 6.4.0. It requires at least version 7.0.0.
~~~

### Implement the read method

In the following, we will use the ipytest package to keep the test in the notebook. For a real new quantity, you would add these tests permanently to the _tests_ directory.

In [ ]:
import ipytest
ipytest.autoconfig()

I recommend writing the tests first, so that you know that the code you write is covered. To do this let's generate some raw data that we can load. For simplicity, we will use the same data that is also in the file.

In [ ]:
from py4vasp import raw
import numpy as np

labels = ("electronic", "ionic")
steps = np.arange(50)
electronic = np.exp(-0.1 * steps)
ionic = 2 * np.exp(-0.2 * steps)
raw_data = raw.Feature(labels, np.array([electronic, ionic]).T)

Next, we generate a simple test to verify that _read_ returns a dictionary with two entries "electronic" and "ionic" that match the raw data.

In [ ]:
%%ipytest -qq

from py4vasp.data import Feature

def test_read():
    feature = Feature.from_data(raw_data)
    actual = feature.read()
    assert np.allclose(actual["electronic"], electronic)
    assert np.allclose(actual["ionic"], ionic)

Of course the test fails, because the class _Feature_ does not exist. The first step is to create the necessary placeholders. Let's do this by adding
~~~python
from py4vasp._data.feature import Feature
~~~
to the [data module](../src/py4vasp/data.py). Then we define a placeholder class
~~~python
class Feature:
    pass
~~~
in the [feature module](../src/py4vasp/_data/feature.py).

If we run the tests again (keep in mind to restart the kernel), the error message changes, because the classmethod _from_data_ is not defined. We can this by modifying the Feature class
~~~python
from py4vasp._data import base

class Feature(base.Refinery):
    "py4vasp requires that all classes are documented."
~~~
This resolves the error with the classmethod. We can also see that the _read_ method is already successfully called.

However, the _read_ method internally calls the _to_dict_ method which the child needs to define. Let's introduce this method in the Feature class
~~~python
def to_dict(self):
    import numpy as np
    labels = self._raw_data.labels
    values = np.array(self._raw_data.values).T
    return { label: data for label, data in zip(labels, values) }
~~~
to make the tests pass. Note where the data comes from: By inheriting from Refinery, Feature obtains the __raw_data_ attribute.

At the moment this only works for the example data. To make it work from the HDF5 file, we need to add a decorator
~~~python
@base.data_access
~~~
to the _to_dict_ method. This decorator does the following things:

* The first time any decorated method is called the relevant HDF5 file is opened. When the method returns the file is closed again. Therefore you can nest decorated methods and it will only open/close the file once.
* It will add an _selection_ argument to the function to decide which quantity is read from the HDF5 file. The __raw_data_ attribute is set to the appropriate one.
* There is logic in place so that the _selection_ works even if the inner function has a selection as well.

Note that the results are only read from the HDF5 file when they are needed. At the time of the request, it will only create a pointer to the relevant dataset. Therefore, you need to load the data explicitly e.g. by accessing a subsection.

The Calculation class automatically adds all Refinery subclasses so that reading from a file or a path works.

In [ ]:
from py4vasp import Calculation

calc = Calculation.from_path(".")
calc.feature.read()

In [ ]:
calc = Calculation.from_file("vaspout.h5")
calc.feature.read()

Notice that the labels are returned as _bytes_ type because the data is stored in this way in the HDF5 file. py4vasp defines some helper functions to deal with this issues in the __util_ module.

### Implement the plot method

Again, we start by defining a simple test case that we make pass afterwards

In [ ]:
import ipytest
ipytest.autoconfig()

In [ ]:
from py4vasp import raw
import numpy as np

labels = ("electronic", "ionic")
steps = np.arange(50)
electronic = np.exp(-0.1 * steps)
ionic = 2 * np.exp(-0.2 * steps)
raw_data = raw.Feature(labels, np.array([electronic, ionic]).T)

In [ ]:
%%ipytest -qq

from py4vasp.data import Feature

def test_plot():
    feature = Feature.from_data(raw_data)
    actual = feature.plot()
    #
    assert actual.xlabel == "step"
    assert actual.ylabel == "energy"
    assert len(actual.series) == 2
    series_electronic, series_ionic = actual.series
    #
    assert series_electronic.name == "electronic"
    assert np.allclose(series_electronic.x, steps)
    assert np.allclose(series_electronic.y, electronic)
    #
    assert series_ionic.name == "ionic"
    assert np.allclose(series_ionic.x, steps)
    assert np.allclose(series_ionic.y, ionic)

py4vasp has a _Graph_ and _Series_ class to wrap around the functionality provided by external plotting libraries. In this way only the relevant subset is supported and it is easier to swap out the current library (plotly) with a different one. All the conversion functionality is added by a _Mixin_ class so that we modify the header of the _Feature_ class [(feature.py)](../src/py4vasp/_data/feature.py) as follows
~~~python
import numpy as np
from py4vasp._data import base
from py4vasp._third_party import graph
from py4vasp._util import convert

class Feature(base.Refinery, graph.Mixin):
    # ...
~~~

This will still fail the test, because we did not define the method _to_graph_ that _Mixin_ expects. Let's introduce it now
~~~python
@base.data_access
def to_graph(self):
    data = self.to_dict()
    steps = np.arange(len(self._raw_data.values))
    series = [
        graph.Series(steps, val, convert.text_to_string(key))
        for key, val in data.items()
    ]
    return graph.Graph(series, xlabel="step", ylabel="energy")
~~~
to make the test pass.

Because of the decorator we can plot the data from the HDF5 file.

In [ ]:
from py4vasp import Calculation

calc = Calculation.from_path(".")
calc.feature.plot()

The _Mixin_ also defines functions to convert the plot to an image

In [ ]:
calc.feature.to_image(filename="feature.png")